Import the full file created by makerecommender.py, and drop extra columns before resaving, so that it has just the geocache code column and the recommender output text.

This is for streamlining the app, but also for keeping the geocaching data a little harder to poach.


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./data/recommendations_file_original.csv')

In [3]:
print(list(df[df['code']=='GC90JDX']['output string'])[0])

There were many caches within the same KMeans cluster, so here are the 5 that are geographically closest:

GC8N2JF: 100 Souvenirs Challenge
GC4NDQP: Color Code: Hello, Yellow!
GC8ZQT3: "Casa de Waffle #9"
GC44MZ3: Snoopy's Library
GCN560: Close to Home Multi

These are all nice and easy and the terrain should be not too bad.

Here are the five caches with the lowest Cosine Distance from your cache. Note that cos dist is always between 0 and 2.

GC7W8J5: Logging Around, cos dist = 0.006, also in the same KMeans cluster
GC8A0RE: 2019 JGC Museum of Southern History, cos dist = 0.012, also in the same KMeans cluster
GC95AZ0: The Road Not Taken, cos dist = 0.018, also in the same KMeans cluster
GC35EEQ: Blue Lake Earl Gilbert Landing, cos dist = 0.028, also in the same KMeans cluster
GC5X7JG: Point The Way, cos dist = 0.028, also in the same KMeans cluster

These are all nice and easy and the terrain should be not too bad.

Thank you, and have fun! Email me at clh@cholland.me if you have an

In [4]:
df.head()

,label,name,code,Y,X,output string
0,1,South River Trail: Triple Claw,GC1A1E7,34.374433,-85.631683,There were many caches within the same KMeans ...
1,49,Canada Traveler Challenge: 5 Provinces,GC92B2K,34.437067,-85.758900,There were many caches within the same KMeans ...
2,47,Milledge Mystery Renewed,GC90JDX,34.435733,-85.754983,There were many caches within the same KMeans ...
3,2,Guarding South Fulton #21,GCH72A,34.688217,-85.526500,There were many caches within the same KMeans ...
4,0,Hottest Park n Ride,GC8D3PX,34.616850,-85.768367,There were many caches within the same KMeans ...


In [5]:
df.drop(columns=['label', 'name', 'Y', 'X'], inplace=True)
df.head()

,code,output string
0,GC1A1E7,There were many caches within the same KMeans ...
1,GC92B2K,There were many caches within the same KMeans ...
2,GC90JDX,There were many caches within the same KMeans ...
3,GCH72A,There were many caches within the same KMeans ...
4,GC8D3PX,There were many caches within the same KMeans ...


In [6]:
print(list(df[df['code']=='GC8D3PX']['output string'])[0])

There were many caches within the same KMeans cluster, so here are the 5 that are geographically closest:

GC7YX3Q: The Great American Cachetime Challenge
GC19YXQ: Electric Wetlands
GC5M2KY: Guarding South Fulton #24
GC6G634: Camouflage
GC1CZ5H: Burn Your Bogie?

These are all nice and easy and the terrain should be not too bad.

Here are the five caches with the lowest Cosine Distance from your cache. Note that cos dist is always between 0 and 2.

GC98DB8: Scared Stupid ( Creepy), cos dist = 0.042, also in the same KMeans cluster
GC8R9V5: What kind of hide is this?, cos dist = 0.05, also in the same KMeans cluster
GC8KNP1: Call Me By Your Name Otto, cos dist = 0.108, also in the same KMeans cluster
GC8Q2Z6: SCT Train Clanking, cos dist = 0.108, also in the same KMeans cluster
GC92BMF: The Lion Roars No More Too, cos dist = 0.121

These are all nice and easy and the terrain should be not too bad.

Thank you, and have fun! Email me at clh@cholland.me if you have any comments or feedback

In [7]:
df.to_csv('./data/recommendations_file.csv',index=False)


I realized in deploying the app that text wasn't being displayed in the correct format, so I needed to parse the bits and pieces back out of the saved output strings. It was MUCH faster to do this than to rerun the file that makes the saved output for each cache (about a minute, vs. 10 hours+).

In [37]:
start_options = []
start_options.append('You picked a somewhat unusual cache - there are no other caches in our 12000+ within the same KMeans cluster.')
start_options.append('Here are all of the caches within the same KMeans cluster:\n')
start_options.append('There were many caches within the same KMeans cluster, so here are the 5 that are geographically closest:\n')

mid_options = []
mid_options.append('\nThese are all nice and easy and the terrain should be not too bad.')
mid_options.append('\nThese are all nice and easy and some have some rough terrain.')
mid_options.append('\nThese are all nice and easy and hiking boots and TecNu may be in order for these.')
mid_options.append('\nSome of these are higher difficulty and the terrain should be not too bad.')
mid_options.append('\nSome of these are higher difficulty and some have some rough terrain.')
mid_options.append('\nSome of these are higher difficulty and hiking boots and TecNu may be in order for these.')
mid_options.append('\nBetter put on your thinking cap for these tough caches and the terrain should be not too bad.')
mid_options.append('\nBetter put on your thinking cap for these tough caches and some have some rough terrain.')
mid_options.append('\nBetter put on your thinking cap for these tough caches and hiking boots and TecNu may be in order for these.')

part2 =  '\n\nHere are the five caches with the lowest Cosine Distance from your cache. Note that cos dist is always between 0 and 2.\n'

ending = '\n\nThank you, and have fun! Email me at clh@cholland.me if you have any comments or feedback.'



In [58]:
import re


In [230]:
def parse_output(str1):

    empty_first=False
    if str1[0]=='Y': 
        empty_first=True

    for n,s in enumerate(start_options):
        L = len(s)
        if str1[0:L] ==s:
            start_no = n
            str1 = str1[L:]
    where = []
    which = []
    if empty_first:
        which.append(-1)
        where.append(0)
    while len(where)<2:
        for n,m in enumerate(mid_options):
            indices = [m.start() for m in re.finditer(mid_options[n], str1)]
            options = [n for j in range(len(indices))]
            where += indices
            which += options
    if empty_first==False:
        temp = pd.DataFrame({'which': which, 'where': where})
        temp = temp.sort_values(by='where')
        which = list(temp['which'])
        where = list(temp['where'])
        omit2nd = [where[1], (where[1]+len(mid_options[which[1]]))]
        str1 = str1[0:omit2nd[0]] + str1[omit2nd[1]:]
        omit1st = [where[0], (where[0]+len(mid_options[which[0]]))]
        str1 = str1[0:omit1st[0]] + str1[omit1st[1]:]
    else: 
        omit2nd = [where[1], (where[1]+len(mid_options[which[1]]))]
        str1 = str1[0:omit2nd[0]] + str1[omit2nd[1]:]
    KMtxt = which[0]
    CDtxt = which[1]
    ndx = str1.find(ending)
    str1 = str1[0:ndx] + str1[(ndx+len(ending)):]
    ndx = str1.find(part2)
    str1 = str1[0:ndx] + str1[(ndx+len(part2)):]
    
    KMcaches = []
    CDcaches = []
    
    caches = str1.split('\n\n')
    if empty_first:
        KMcaches = []
        CDcaches = caches[0].split('\n')
        CDcaches = [c for c in CDcaches if c!='']

    else:
        KMcaches = caches[0].split('\n')
        CDcaches = caches[1].split('\n')
        KMcaches = [c for c in KMcaches if c!='']
        CDcaches = [c for c in CDcaches if c!='']
    N_KM = len(KMcaches)

    return start_no, KMtxt, CDtxt, KMcaches, CDcaches, str1, N_KM

# God bless StackOverflow, really:
# https://stackoverflow.com/questions/4664850/how-to-find-all-occurrences-of-a-substring

In [232]:
starts = []
KMtxts = []
CDtxts = []
KM0 = []
KM1 = []
KM2 = []
KM3 = []
KM4 = []
CD0 = []
CD1 = []
CD2 = []
CD3 = []
CD4 = []
numKM= []

for n,c in enumerate(codes):
    start_no, KMtxt, CDtxt, KMcaches, CDcaches, str1, N_KM = parse_output(outpts[n])
    while len(KMcaches)<5: KMcaches.append('')
    starts.append(start_no)
    KMtxts.append(KMtxt)
    CDtxts.append(CDtxt)
    KM0.append(KMcaches[0])
    KM1.append(KMcaches[1])
    KM2.append(KMcaches[2])
    KM3.append(KMcaches[3])
    KM4.append(KMcaches[4])
    CD0.append(CDcaches[0])
    CD1.append(CDcaches[1])
    CD2.append(CDcaches[2])
    CD3.append(CDcaches[3])
    CD4.append(CDcaches[4])
    numKM.append(N_KM)
    if n%500==0: print(f'Parsed {n+1} of 12433, {(n+1)*100/12433}% completed...')


Parsed 1 of 12433, 0.008043111075363952% completed...
Parsed 501 of 12433, 4.029598648757339% completed...
Parsed 1001 of 12433, 8.051154186439314% completed...
Parsed 1501 of 12433, 12.07270972412129% completed...
Parsed 2001 of 12433, 16.094265261803265% completed...
Parsed 2501 of 12433, 20.115820799485242% completed...
Parsed 3001 of 12433, 24.137376337167215% completed...
Parsed 3501 of 12433, 28.15893187484919% completed...
Parsed 4001 of 12433, 32.180487412531164% completed...
Parsed 4501 of 12433, 36.20204295021314% completed...
Parsed 5001 of 12433, 40.22359848789512% completed...
Parsed 5501 of 12433, 44.245154025577094% completed...
Parsed 6001 of 12433, 48.26670956325907% completed...
Parsed 6501 of 12433, 52.28826510094105% completed...
Parsed 7001 of 12433, 56.309820638623016% completed...
Parsed 7501 of 12433, 60.33137617630499% completed...
Parsed 8001 of 12433, 64.35293171398698% completed...
Parsed 8501 of 12433, 68.37448725166895% completed...
Parsed 9001 of 12433, 7

In [234]:


recs = pd.DataFrame({
                        'code': codes,
                        'startoption': starts,
                        'KMtxtoption': KMtxts,
                        'KMcache0': KM0,
                        'KMcache1': KM1,
                        'KMcache2': KM2,
                        'KMcache3': KM3,
                        'KMcache4': KM4,
                        'CDtxtoption': CDtxts,
                        'CDcache0': CD0,
                        'CDcache1': CD1,
                        'CDcache2': CD2,
                        'CDcache3': CD3,
                        'CDcache4': CD4,
                        'num KM': numKM
                    })

recs.head(2)

,code,startoption,KMtxtoption,KMcache0,KMcache1,KMcache2,KMcache3,KMcache4,CDtxtoption,CDcache0,CDcache1,CDcache2,CDcache3,CDcache4,num KM
0,GC1A1E7,2,0,GC44KZ6: (Not THAT) Beaver Cache,GC44NXP: SRT: UTP,GC11HRX: B and B Little Chicken,GC1W1YB: Black Cat Tips,GC5VXDK: Under the Boardwalk,0,"GC2Q4CD: Georgia I-95 Solar System - Rhea, cos...","GC8ZQ7E: Hello I'm Johnnie's Cache, cos dist =...","GC2ZG54: All Roads Lead to Rome Iter VIII, cos...","GC8A5MG: Relic, cos dist = 0.03, also in the s...","GC4M1E2: Greek Redux - Omega, cos dist = 0.03,...",5
1,GC92B2K,2,0,GC8N2GW: Linecrest One,GC23RM9: Publix Enemy #18,GC75A8Z: Fayetteville. Commons,GC8D3RK: World Traveler Challenge: 3 Continents,GCV92H: Phoenix,0,"GC95RNV: Zone G, cos dist = 0.02, also in the ...","GC7GK2M: Hiking2175's Golden Ammo Can, cos dis...","GC4AZ8Z: Kayak Series State Line Island, cos d...","GC4278Y: On the Mark, cos dist = 0.086, also i...","GC87WTP: RS #85: Like a Rolling Stone, cos dis...",5


In [246]:
recs.loc[35]

code                                                     GC3BG9G
startoption                                                    2
KMtxtoption                                                    0
KMcache0        GC1RDZK: Fields of McCurry Park AL - Bonus Cache
KMcache1                                     GC1W4YG: The Master
KMcache2                           GC101M3: Gillian & Ella Cache
KMcache3                                GCM9W8: Yellow Submarine
KMcache4       GC8M840: The Cache Above the City Beneath the ...
CDtxtoption                                                    6
CDcache0       GC6ZGGY: Guarding South Fulton #27, cos dist =...
CDcache1       GC4KGMH: a "Gin"  joint next to the tracks?, c...
CDcache2       GC513MK: Baby Bird Is Lost..., cos dist = 0.01...
CDcache3       GC43DVP: A Horse with No Name - America, cos d...
CDcache4       GC4M3M7: Orthoguy This Ones' For You, cos dist...
num KM                                                         5
Name: 35, dtype: object

In [247]:
recs.loc[4958]

code                                                     GC8J18J
startoption                                                    0
KMtxtoption                                                   -1
KMcache0                                                        
KMcache1                                                        
KMcache2                                                        
KMcache3                                                        
KMcache4                                                        
CDtxtoption                                                    0
CDcache0       GC35EH8: Don't Poke the Bear #1, cos dist = 0.953
CDcache1       GC2358Z: Day of an American Battle-21JUL1861-F...
CDcache2                     GC8H9NQ: The Path, cos dist = 0.957
CDcache3       GC8M4QX: Christmas Visit from Santa, cos dist ...
CDcache4       GC7K9YV: By the river and through the woods, c...
num KM                                                         0
Name: 4958, dtype: object

In [258]:
recs.to_csv('./data/recommendations_file_original.csv',index=False)


In [254]:
recs = pd.read_csv('./data/recommendations_file.csv')

In [257]:
list(recs[recs['code']=='GC5GECD']['CDcache0'])[0]

'GC5GECD:  "Let\'s Play Ball!" #1, cos dist = 0.0, also in the same KMeans cluster'